In [ ]:
import sys, os, iminuit, h5py, time
import numpy as np
from scipy import stats

# Defining the basics of the parameter file

Here we will set up a parameter file for Perseus. We start with a redshift for the cluster and with a list of empty upper and lower bounds which will be assigned as relevant for the model components.

In [ ]:
z = 0.016 # the best-fit redshift for Perseus

upper_bounds = []
lower_bounds = []

# Define the continuum model

We allow for arbitrary combinations of `nlapec`, `folded_powerlaw`, and `unfolded_powerlaw` models. In the following, we define the model to include 2 `nlapec` continua, 1 `folded_powerlaw`, and 1 `unfolded_powerlaw`. We also must define bounds on the parameters for these models for use in global optimization.

Continuum components are defined by a normalization and a shape parameter. For each of the three, the normalization parameter $N$ defines the fraction of the observed counts predicted by the given continuum component. This is a particularly convenient variable to set sensible bounds on and optimize over. We allow this normalization parameter to vary between `0` and `1.1`.

For the `nlapec`, the shape parameter is the temperature, which is allowed to freely vary in the interval `[1, 12]` Kelvin. Each of the power-laws have take an index as shape parameter, which is allowed to vary within `[-4, 1]`.


We also specify the optimization bounds for the hydrogen absorption. The parameter over which we optimize is the `Log22` which is the $\log_{10}$ of the hydrogen line depth in units of $10^{22} /\mathrm{cm}^2$.

In [ ]:
upper_bounds.append(1)
lower_bounds.append(-1)

num_nlapec = 2
num_folded_pl = 1
num_unfolded_pl = 1

for i in range(num_nlapec):
    
    # Normalization bounds
    upper_bounds.append(1.1)
    lower_bounds.append(0)
    
    # Temperature bounds
    lower_bounds.append(1)
    upper_bounds.append(12)
    
    
for i in range(num_folded_pl):
    
    # Index bounds
    upper_bounds.append(1)
    lower_bounds.append(-4)
    
    # Normalization bounds
    upper_bounds.append(1.1)
    lower_bounds.append(0) 
    
for i in range(num_unfolded_pl):
    
    # Index bounds
    upper_bounds.append(1)
    lower_bounds.append(-4)
    
    # Normalization bounds
    upper_bounds.append(1.1)
    lower_bounds.append(0) 

# Define the candidate line list and bounds

In this example, we will set up an optimization which allows for the line flux, the line rest energy, and the line width to vary.


We start by defining the list of lines which will be included in the model. The list of lines is defined by a set of emission-frame energies and associated boolean, with `True` for redshifted astrophysical lines or `False` for unredshifted instrumental lines. In this analysis, we do not use any instrumental lines.

In [ ]:
# Line energies in keV
line_energies = np.array([3.124, 3.32, 3.472, 3.511, 3.617, 3.685, 3.705, 3.861, 3.902, 3.936, 4.107, 4.584, 5.682])
line_types = np.ones((len(line_energies)), dtype = bool)

Lines are parametrized by a displacement from their expected rest energy $\Delta E$, a flux $F$ and a width parameter $\sigma_E$.
- Lines are allowed to vary in location up to $\pm 5$ eV. 
- The line flux parameter, in units of $10^{-6}$ photons/cm$^2$/s is allowed to vary over an effectively unbounded positive interval, with the exception of lines in the vicinity of the 3.57 keV line, for which we adopt upper bounds following the treatment of https://arxiv.org/abs/1402.2301.
- The line width is allowed to vary between [$10^{-4}$, $10^{-2}$] $\times E_{line}$.

We now loop over the list of lines, adding their parameters to our bounds lists.

In [ ]:
# Making the DeltaE bounds
for i in range(len(line_energies)):
    upper_bounds.append(5e-3) # +5eV
    lower_bounds.append(5e-3) # -5eV

# Making the Flux bounds
for i in range(len(line_energies)):
    if line_energies[i] == 3.472:
        upper_bounds.append(5.55)
        
    elif line_energies[i] == 3.511:
        upper_bounds.append(13.71)
        
    elif line_energies[i] == 3.617:
        upper_bounds.append(1.92)
        
    elif line_energies[i] == 3.685:
        upper_bounds.append(45.3)
        
    elif line_energies[i] == 3.705:
        upper_bounds.append(34.8)
        
    else:
        upper_bounds.append(400)
        
    lower_bounds.append(0)
    
# Making the SigmaE bounds
for i in range(len(line_energies)):
    upper_bounds.append(1e-2 * line_energies[i])
    lower_bounds.append(1e-4 * line_energies[i])

# Save the model

We have now fully specified the model. We will save it in a format that our analysis code can make use of.

In [ ]:
fit_dict = dict()

fit_dict['z'] = z
fit_dict['lines'] = line_energies
fit_dict['redshifts'] = line_types
fit_dict['num_apec']  = num_nlapec
fit_dict['num_folded'] = num_folded_pl
fit_dict['num_unfolded'] = num_unfolded_pl

fit_dict['upper'] = upper_bounds
fit_dict['lower'] = lower_bounds

np.savez('./Fitting/E3/Perseus/Initial.npz', **fit_dict)